In [1]:
import numpy as np
import pandas as pd

filename = '../npz/BTCUSD_1T.npz'
data = np.load(filename, allow_pickle=True)['data']
# преобразование в формат pandas
df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# Преобразование 'timestamp' в datetime и установка его как индекс DataFrame
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Преобразование всех столбцов с ценами и объемом в числовой формат
for column in ['open', 'high', 'low', 'close', 'volume']:
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [2]:
from flask import Flask, render_template, jsonify, request
import threading
import os
import numpy as np

# Получаем абсолютный путь к текущей директории, в которой находится Jupyter Notebook
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
static_folder = os.path.join(notebook_dir, 'static')
template_folder = os.path.join(notebook_dir, 'templates')

app = Flask(__name__, static_folder=static_folder, template_folder=template_folder)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/ohlc-data', methods=['GET'])
def get_ohlc_data():
    len = 40
    return jsonify({ 'close': df['close'][:len].to_list(), 'vol': df['volume'][:len].to_list() })

@app.route('/server_info')
def server_info():
    server_software = request.environ.get('SERVER_SOFTWARE', '')
    return f"Server Software: {server_software}"

# Функция для запуска Flask в потоке
def run_flask():
    app.run(port=8050, debug=True, use_reloader=False)

# Запуск Flask в отдельном потоке
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [19/Aug/2024 08:50:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2024 08:50:11] "GET /static/main.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2024 08:50:11] "GET /ohlc-data HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2024 08:59:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2024 08:59:39] "GET /static/main.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2024 08:59:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2024 08:59:50] "GET /static/main.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2024 08:59:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2024 08:59:56] "GET /static/main.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2024 09:01:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2024 09:01:10] "GET /static/main.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2024 09:01:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2024 09:01:14] "GET /static/main.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2024 09:03:40] "GET / HTTP/1.1" 200 -
127.0.0.1 

In [3]:
from IPython.display import IFrame

# Встраиваем страницу Flask в Jupyter Notebook
IFrame(src="http://localhost:8050", width=660, height=500)
